In [13]:
import requests
import torch
from PIL import Image
from io import BytesIO

from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cpu"

# Note that passing the image urls (instead of the actual pil images) to the processor is also possible
image1 = load_image("target-Photoroom.png-Photoroom.png")
# image2 = load_image("https://cdn.britannica.com/59/94459-050-DBA42467/Skyline-Chicago.jpg")
# image3 = load_image("https://cdn.britannica.com/68/170868-050-8DDE8263/Golden-Gate-Bridge-San-Francisco.jpg")


In [15]:
import transformers
print(transformers.__version__)



4.32.0.dev0


In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="HuggingFaceM4/idefics2-8b")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/adol/virtualenvs/pip_only/lib/python3.9/site-packages/transformers/utils/import_utils.py", line 1125, in _get_module
  File "/home/adol/anaconda3/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/adol/virtualenvs/pip_only/lib/python3.9/site-packages/transformers/pipelines/__init__.py", line 30, in <module>
    from ..models.auto.modeling_auto import AutoModelForDepthEstimation, AutoModelForImageToImage
ImportError: cannot

In [11]:
# Load model directly
from transformers import AutoProcessor, AutoModelForPreTraining

processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")
model = AutoModelForPreTraining.from_pretrained("HuggingFaceM4/idefics2-8b")

# Create inputs
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What do we see in this image?"},
        ]
    },
    # {
    #     "role": "assistant",
    #     "content": [
    #         {"type": "text", "text": "In this image, we can see the city of New York, and more specifically the Statue of Liberty."},
    #     ]
    # },
    # {
    #     "role": "user",
    #     "content": [
    #         {"type": "image"},
    #         {"type": "text", "text": "And how about this image?"},
    #     ]
    # },       
]
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}


# Generate
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_texts)
# ['User: What do we see in this image? \nAssistant: In this image, we can see the city of New York, and more specifically the Statue of Liberty. \nUser: And how about this image? \nAssistant: In this image we can see buildings, trees, lights, water and sky.']


ValueError: Unrecognized processing class in HuggingFaceM4/idefics2-8b. Can't instantiate a processor, a tokenizer, an image processor or a feature extractor for this model. Make sure the repository containsthe files of at least one of those processing classes.

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/recurrentgemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/recurrentgemma-2b", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cpu")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))



OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/google/recurrentgemma-2b and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.